# Connections using ConceptNet

The goal is to pick a term for a category, then to pick a set of terms that all share some (perhaps the same) relation with the former.

In [5]:
import requests
import time

In [59]:
def conceptnet(term: str, lang:str='en', all_results=True, verbose=False) -> dict:
    if verbose:
        print(f'Querying term "{term}"')
        start = time.time()
    res = requests.get(f'http://api.conceptnet.io/c/{lang}/{term}')
    if verbose:
        elapsed = time.time() - start
        print(f'Response received after {elapsed}s.')
    return res.json()



In [20]:
word = conceptnet('word')

In [58]:
for edge in word['edges'] + word2['edges']:
    start = edge['start']['label']
    end = edge['end']['label']
    rel = edge['rel']['label']
    print(f'{start} {rel} {end}')

a word PartOf a sentence
a verb IsA a word
a letter PartOf a word
A word PartOf language
word RelatedTo sentence
a word CapableOf mean things
earth IsA a word
a word CapableOf mean more than one thing
a word IsA a piece of a sentence
語 Synonym word
word RelatedTo language
word RelatedTo unit
word RelatedTo part
four-letter word IsA word
terminology IsA word
metonym IsA word
form IsA word
word RelatedTo scripture
synonym IsA word
loanword IsA word
poručivati RelatedTo word
terme Synonym word
oxytone IsA word
janji Synonym word
primitive IsA word
λόγοσ RelatedTo word
terminology IsA word
arahan Synonym word
単語 Synonym word
word Synonym reč
loanword IsA word
word RelatedTo scripture
anagram IsA word
เวิร์ด Synonym word
antonym IsA word
fjale Synonym word
form IsA word
affix PartOf word
hypernym IsA word
metonym IsA word


In [40]:
VALID_RELS = (
    'PartOf',
    'IsA',
    # 'CapableOf'
)

In [46]:
word['view']

{'@id': '/c/en/word?offset=0&limit=20',
 '@type': 'PartialCollectionView',
 'comment': "There are more results. Follow the 'nextPage' link for more.",
 'firstPage': '/c/en/word?offset=0&limit=20',
 'nextPage': '/c/en/word?offset=20&limit=20',
 'paginatedProperty': 'edges'}

In [55]:
word2 = requests.get('http://api.conceptnet.io/c/en/word?offset=20&limit=20').json()

In [57]:
word2['view']

{'@id': '/c/en/word?offset=20&limit=20',
 '@type': 'PartialCollectionView',
 'comment': "There are more results. Follow the 'nextPage' link for more.",
 'firstPage': '/c/en/word?offset=0&limit=20',
 'nextPage': '/c/en/word?offset=40&limit=20',
 'paginatedProperty': 'edges',
 'previousPage': '/c/en/word?offset=0&limit=20'}